In [0]:
%sql
--information about all the teams in eurocup
CREATE or replace TABLE teamsinfo10(
    TeamID INT,
    TeamName VARCHAR(50) NOT NULL,
    GroupName CHAR(1) NOT NULL
);

In [0]:
%sql
--information inserting values to the table

INSERT INTO teamsinfo10(TeamID, TeamName, GroupName) VALUES
(1, 'France', 'A'),
(2, 'Portugal', 'A'),
(3, 'Germany', 'A'),
(4, 'Hungary', 'A'),
(5, 'Spain', 'B'),
(6, 'Sweden', 'B'),
(7, 'Poland', 'B'),
(8, 'Slovakia', 'B'),
(9, 'Italy', 'C'),
(10, 'Switzerland', 'C'),
(11, 'Turkey', 'C'),
(12, 'Wales', 'C'),
(13, 'Belgium', 'D'),
(14, 'Denmark', 'D'),
(15, 'Russia', 'D'),
(16, 'Finland', 'D'),
(17, 'Netherlands', 'E'),
(18, 'Ukraine', 'E'),
(19, 'Austria', 'E'),
(20, 'North Macedonia', 'E'),
(21, 'England', 'F'),
(22, 'Croatia', 'F'),
(23, 'Scotland', 'F'),
(24, 'Czech Republic', 'F');

select * 
from teamsinfo10

TeamID,TeamName,GroupName
1,France,A
2,Portugal,A
3,Germany,A
4,Hungary,A
5,Spain,B
6,Sweden,B
7,Poland,B
8,Slovakia,B
9,Italy,C
10,Switzerland,C


In [0]:
%sql
--this table contains information about all the matches that took place
CREATE TABLE matchinfo1(
    matchid INT,
    team1 VARCHAR(30),
    team2 VARCHAR(30),
    team1score INTEGER,
    team2score INTEGER,
    result VARCHAR(30),
    stage VARCHAR (255)
);

In [0]:
%sql

insert into matchinfo1(matchid, team1, team2, team1score, team2score, result, stage)
SELECT
    ROW_NUMBER() OVER (ORDER BY t1.TeamId) AS matchid,
    t1.TeamName AS team1,
    t2.TeamName AS team2,
    FLOOR(RAND()*(6)+1) AS team1Score,
    FLOOR(RAND()*(6)+1) AS team2Score,
    NULL AS Result,
    'Group Stage' as stage
FROM teamsinfo10 t1
join teamsinfo10 t2 ON t1.GroupName = t2.GroupName AND t1.TeamId < t2.TeamId;

UPDATE matchinfo1
SET result = CASE
    WHEN team1score > team2score THEN team1
    WHEN team1score < team2score THEN team2
    ELSE 'Draw'
END;


select *
from matchinfo1

matchid,team1,team2,team1score,team2score,result,stage
1,France,Hungary,5,3,France,Group Stage
2,France,Germany,2,3,Germany,Group Stage
3,France,Portugal,1,4,Portugal,Group Stage
4,Portugal,Hungary,5,5,Draw,Group Stage
5,Portugal,Germany,1,2,Germany,Group Stage
6,Germany,Hungary,3,4,Hungary,Group Stage
7,Spain,Slovakia,6,2,Spain,Group Stage
8,Spain,Poland,6,2,Spain,Group Stage
9,Spain,Sweden,2,6,Sweden,Group Stage
10,Sweden,Slovakia,5,5,Draw,Group Stage


In [0]:
%sql
-- a table to store all match related stats
CREATE TABLE teamstats1 AS
SELECT
    t.TeamName,
    t.GroupName,
    COUNT(CASE WHEN m.result = t.TeamName THEN 1 END) AS Wins,
    COUNT(CASE WHEN m.result = 'Draw' THEN 1 END) AS Draws,
    COUNT(CASE WHEN m.result != t.TeamName AND m.result != 'Draw' THEN 1 END) AS Losses,
    SUM(CASE WHEN m.result = t.TeamName THEN 3 WHEN m.result = 'Draw' THEN 1 ELSE 0 END) AS TotalPoints,
    COUNT(*) AS TotalGamesPlayed,
    SUM(CASE WHEN t.TeamName = m.team1 THEN m.team1score ELSE m.team2score END) AS GoalsFor,
    SUM(CASE WHEN t.TeamName = m.team1 THEN m.team2score ELSE m.team1score END) AS GoalsAgainst,
    SUM(CASE WHEN t.TeamName = m.team1 THEN m.team1score ELSE m.team2score END) -
    SUM(CASE WHEN t.TeamName = m.team1 THEN m.team2score ELSE m.team1score END) AS GoalDifference
FROM
    teamsinfo10 t
JOIN
    matchinfo1 m ON t.TeamName = m.team1 OR t.TeamName = m.team2
GROUP BY
    t.GroupName,t.TeamName
order by t.GroupName;

select* 
from teamstats1

TeamName,GroupName,Wins,Draws,Losses,TotalPoints,TotalGamesPlayed,GoalsFor,GoalsAgainst,GoalDifference
Germany,A,2,0,1,6,3,8,7,1
Hungary,A,1,1,1,4,3,12,13,-1
France,A,1,0,2,3,3,8,10,-2
Portugal,A,1,1,1,4,3,10,8,2
Slovakia,B,1,1,1,4,3,10,13,-3
Sweden,B,2,1,0,7,3,16,10,6
Spain,B,2,0,1,6,3,14,10,4
Poland,B,0,0,3,0,3,7,14,-7
Switzerland,C,2,0,1,6,3,11,12,-1
Turkey,C,2,0,1,6,3,11,9,2


In [0]:
%sql
-- a temporary view table for ranking
CREATE OR Replace TEMPORARY VIEW temp_ranking AS
SELECT
    ts.TeamName,
    ts.Wins,
    ts.Draws,
    ts.Losses,
    ts.TotalPoints,
    ts.TotalGamesPlayed,
    ts.GoalsFor,
    ts.GoalsAgainst,
    ts.GoalDifference,
    t.GroupName,
    RANK() OVER (PARTITION BY t.GroupName ORDER BY ts.TotalPoints DESC, ts.GoalDifference DESC, ts.GoalsFor DESC) AS rank
FROM
    teamstats1 ts
JOIN
    teamsinfo10 t ON ts.TeamName = t.TeamName;


SELECT * FROM temp_ranking;

TeamName,Wins,Draws,Losses,TotalPoints,TotalGamesPlayed,GoalsFor,GoalsAgainst,GoalDifference,GroupName,rank
Germany,2,0,1,6,3,8,7,1,A,1
Portugal,1,1,1,4,3,10,8,2,A,2
Hungary,1,1,1,4,3,12,13,-1,A,3
France,1,0,2,3,3,8,10,-2,A,4
Sweden,2,1,0,7,3,16,10,6,B,1
Spain,2,0,1,6,3,14,10,4,B,2
Slovakia,1,1,1,4,3,10,13,-3,B,3
Poland,0,0,3,0,3,7,14,-7,B,4
Turkey,2,0,1,6,3,11,9,2,C,1
Switzerland,2,0,1,6,3,11,12,-1,C,2


In [0]:
%sql
--table to store all 16 teams
CREATE TABLE roundof16 (
  TeamName VARCHAR(255),
  GroupName VARCHAR(255),
  rank INTEGER
);

In [0]:
%sql
Insert into roundof16 
    SELECT
        t.TeamName,
        t.GroupName,
        r.rank
    FROM
        teamsinfo10 t
        JOIN temp_ranking r ON t.TeamName = r.TeamName
    WHERE
        r.rank <= 2; 

num_affected_rows,num_inserted_rows
12,12


In [0]:
%sql
Insert into roundof16 
    SELECT
        t.TeamName,
        t.GroupName,
        r.rank
    FROM
        teamsinfo10 t
        JOIN temp_ranking r ON t.TeamName = r.TeamName
    WHERE
        r.rank = 3
        limit 4;


select *
from roundof16

TeamName,GroupName,rank
Germany,A,1
Portugal,A,2
Sweden,B,1
Spain,B,2
Turkey,C,1
Switzerland,C,2
Denmark,D,1
Belgium,D,2
North Macedonia,E,1
Netherlands,E,2


In [0]:
%sql
CREATE TABLE ThirdPlaceCombination1(
  ID INTEGER,
  SelectedGroup VARCHAR(4),
  Team1B VARCHAR(1),
  Team1C VARCHAR(1),
  Team1E VARCHAR(1),
  Team1F VARCHAR(1)
);


In [0]:
%sql
INSERT INTO  ThirdPlaceCombination1 (ID, SelectedGroup, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

SELECT * FROM ThirdPlaceCombination1;

ID,SelectedGroup,Team1B,Team1C,Team1E,Team1F
1,ABCD,A,D,B,C
2,ABCE,A,E,B,C
3,ABCF,A,F,B,C
4,ABDE,D,E,A,B
5,ABDF,D,F,A,B
6,ABEF,E,F,B,A
7,ACDE,E,D,C,A
8,ACDF,F,D,C,A
9,ACEF,E,F,C,A
10,ADEF,E,F,D,A


In [0]:
%sql
INSERT INTO matchinfo1(matchid,
team1,
team2,
team1score,
team2score,
result,
stage)
WITH Combination AS (
    SELECT CONCAT_WS('', sort_array(Collect_list(GroupName))) AS GroupCombination
    FROM roundof16 r1
    WHERE rank = 3 
),
keys AS (
    SELECT * FROM ThirdPlaceCombination1
    WHERE selectedgroup = (SELECT GroupCombination FROM Combination)
)
SELECT 
    37 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'A' AND r1.rank = 1 AND r2.GroupName = 'C' AND r2.rank = 2
    

UNION ALL

SELECT 
    38 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'A' AND r1.rank = 2 AND r2.GroupName = 'B' AND r2.rank = 2



UNION ALL

SELECT 
    39 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'B' AND r1.rank = 1 AND r2.GroupName = (SELECT Team1B FROM keys) AND r2.rank = 3

  

UNION ALL

SELECT 
    40 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'C' AND r1.rank = 1 AND r2.GroupName = (SELECT Team1C FROM keys) AND r2.rank = 3
 
    

UNION ALL

SELECT 
    41 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'F' AND r1.rank = 1 AND r2.GroupName = (SELECT Team1F FROM keys) AND r2.rank = 3

    

UNION ALL

SELECT 
    42 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'D' AND r1.rank = 2 AND r2.GroupName = 'E' AND r2.rank = 2


UNION ALL

SELECT 
    43 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'E' AND r1.rank = 1 AND r2.GroupName = (select Team1E from keys) AND r2.rank = 3


UNION ALL

SELECT 
    44 AS matchid,
    r1.TeamName AS team1,
    r2.TeamName AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Round of 16' AS stage
FROM 
    roundof16 r1
JOIN 
  roundof16 r2 ON r1.GroupName = 'D' AND r1.rank = 2 AND r2.GroupName = 'F' AND r2.rank = 2

num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
UPDATE matchinfo1
SET result = CASE
    WHEN team1score > team2score THEN team1
    WHEN team1score < team2score THEN team2
END
WHERE stage = "Round of 16";


select *
from matchinfo1
order by matchid

matchid,team1,team2,team1score,team2score,result,stage
1,France,Hungary,5,3,France,Group Stage
2,France,Germany,2,3,Germany,Group Stage
3,France,Portugal,1,4,Portugal,Group Stage
4,Portugal,Hungary,5,5,Draw,Group Stage
5,Portugal,Germany,1,2,Germany,Group Stage
6,Germany,Hungary,3,4,Hungary,Group Stage
7,Spain,Slovakia,6,2,Spain,Group Stage
8,Spain,Poland,6,2,Spain,Group Stage
9,Spain,Sweden,2,6,Sweden,Group Stage
10,Sweden,Slovakia,5,5,Draw,Group Stage


In [0]:
%sql
INSERT INTO matchinfo1 (matchid,
team1,
team2,
team1score,
team2score,
result,
stage)
SELECT 
    45 AS matchid,
    i1.team1 AS team1,
    i2.team2 AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Quarter-finals' AS stage
FROM 
    matchinfo1 i1
JOIN 
    matchinfo1 i2 ON 
        i1.matchid = 39 AND i2.matchid = 37

UNION ALL

SELECT 
    46 AS matchid,
    i1.team1 AS team1,
    i2.team2 AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Quarter-finals' AS stage
FROM 
    matchinfo1 i1
JOIN 
    matchinfo1 i2 ON 
        i1.matchid = 41 AND i2.matchid = 42
      

UNION ALL

SELECT 
    47 AS matchid,
    i1.team1 AS team1,
    i2.team2 AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Quarter-finals' AS stage
FROM 
    matchinfo1 i1
JOIN 
    matchinfo1 i2 ON 
        i1.matchid = 40 AND i2.matchid = 38
       

UNION ALL

SELECT 
    48 AS matchid,
    i1.team1 AS team1,
    i2.team2 AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Quarter-finals' AS stage
FROM 
    matchinfo1 i1
JOIN 
    matchinfo1 i2 ON 
        i1.matchid = 43 AND i2.matchid = 44
        

num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
UPDATE matchinfo1
SET result = CASE
    WHEN team1score > team2score THEN team1
    WHEN team1score < team2score THEN team2
END
WHERE stage = "Quarter-finals";


select *
from matchinfo1
order by matchid

matchid,team1,team2,team1score,team2score,result,stage
1,France,Hungary,5,3,France,Group Stage
2,France,Germany,2,3,Germany,Group Stage
3,France,Portugal,1,4,Portugal,Group Stage
4,Portugal,Hungary,5,5,Draw,Group Stage
5,Portugal,Germany,1,2,Germany,Group Stage
6,Germany,Hungary,3,4,Hungary,Group Stage
7,Spain,Slovakia,6,2,Spain,Group Stage
8,Spain,Poland,6,2,Spain,Group Stage
9,Spain,Sweden,2,6,Sweden,Group Stage
10,Sweden,Slovakia,5,5,Draw,Group Stage


In [0]:
%sql
INSERT INTO matchinfo1 (matchid,
team1,
team2,
team1score,
team2score,
result,
stage)
SELECT 
    49 AS matchid,
    i1.team1 AS team1,
    i2.team2 AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Semi-finals' AS stage
FROM 
    matchinfo1 i1
JOIN 
    matchinfo1 i2 ON 
        i1.matchid = 45 AND i2.matchid = 46

  UNION ALL

  SELECT 
    50 AS matchid,
    i1.team1 AS team1,
    i2.team2 AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Semi-finals' AS stage
FROM 
    matchinfo1 i1
JOIN 
    matchinfo1 i2 ON 
        i1.matchid = 47 AND i2.matchid = 48

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
select * from matchinfo1

matchid,team1,team2,team1score,team2score,result,stage
1,France,Hungary,5,3,France,Group Stage
2,France,Germany,2,3,Germany,Group Stage
3,France,Portugal,1,4,Portugal,Group Stage
4,Portugal,Hungary,5,5,Draw,Group Stage
5,Portugal,Germany,1,2,Germany,Group Stage
6,Germany,Hungary,3,4,Hungary,Group Stage
7,Spain,Slovakia,6,2,Spain,Group Stage
8,Spain,Poland,6,2,Spain,Group Stage
9,Spain,Sweden,2,6,Sweden,Group Stage
10,Sweden,Slovakia,5,5,Draw,Group Stage


In [0]:
%sql
INSERT INTO matchinfo1 (matchid,
team1,
team2,
team1score,
team2score,
result,
stage)
WITH ThirdPositions1 AS (
    SELECT
        CASE
            WHEN result!= team1 THEN team1
            ELSE team2
        END AS ThirdPosition
    FROM
        matchinfo1
    WHERE 
        stage = 'Semi-finals'
)

SELECT 
    51 AS matchid,
    tp1.ThirdPosition AS team1,
    tp2.ThirdPosition AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Third Position' AS stage
    
FROM 
    ThirdPositions1 tp1
JOIN 
    ThirdPositions1 tp2 ON tp1.ThirdPosition < tp2.ThirdPosition




num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE matchinfo1
SET result = CASE
    WHEN team1score > team2score THEN team1
    WHEN team1score < team2score THEN team2
END
WHERE stage = "Third Position";


select *
from matchinfo1
order by matchid

matchid,team1,team2,team1score,team2score,result,stage
1,France,Hungary,5,3,France,Group Stage
2,France,Germany,2,3,Germany,Group Stage
3,France,Portugal,1,4,Portugal,Group Stage
4,Portugal,Hungary,5,5,Draw,Group Stage
5,Portugal,Germany,1,2,Germany,Group Stage
6,Germany,Hungary,3,4,Hungary,Group Stage
7,Spain,Slovakia,6,2,Spain,Group Stage
8,Spain,Poland,6,2,Spain,Group Stage
9,Spain,Sweden,2,6,Sweden,Group Stage
10,Sweden,Slovakia,5,5,Draw,Group Stage


In [0]:
%sql
INSERT INTO matchinfo1 (matchid,
team1,
team2,
team1score,
team2score,
result,
stage)
SELECT 
    52 AS matchid,
    i1.team1 AS team1,
    i2.team2 AS team2,
    CAST(rand() * 6 AS INT) AS team1score, 
    CAST(rand() * 6 AS INT) AS team2score,
    'Null' AS result,
    'Final' AS stage
FROM 
    matchinfo1 i1
JOIN 
    matchinfo1 i2 ON 
        i1.matchid = 49 AND i2.matchid = 50

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql

UPDATE matchinfo1
SET result = CASE
    WHEN team1score > team2score THEN team1
    WHEN team1score < team2score THEN team2
        ELSE 
          CASE 
                WHEN (team1score + 1) % 2 = 1 THEN team1
                ELSE team2
            END
    END



num_affected_rows
52


In [0]:
%sql
select *
from matchinfo1

matchid,team1,team2,team1score,team2score,result,stage
1,France,Hungary,5,3,France,Group Stage
2,France,Germany,2,3,Germany,Group Stage
3,France,Portugal,1,4,Portugal,Group Stage
4,Portugal,Hungary,5,5,Hungary,Group Stage
5,Portugal,Germany,1,2,Germany,Group Stage
6,Germany,Hungary,3,4,Hungary,Group Stage
7,Spain,Slovakia,6,2,Spain,Group Stage
8,Spain,Poland,6,2,Spain,Group Stage
9,Spain,Sweden,2,6,Sweden,Group Stage
10,Sweden,Slovakia,5,5,Slovakia,Group Stage


In [0]:
%python
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("visulization") \
    .getOrCreate()

In [0]:
%python
gs = spark.sql("SELECT * from teamstats1")
qf = spark.sql("SELECT team1,team2, team1score,team2score FROM matchinfo1 WHERE stage = 'Quarter-finals' ORDER BY matchid")
r16 = spark.sql("SELECT team1,team2, team1score,team2score FROM matchinfo1 WHERE stage = 'Round of 16' ORDER BY matchid")
sf = spark.sql("SELECT team1,team2, team1score,team2score FROM matchinfo1 WHERE stage = 'Semi-finals' ORDER BY matchid desc")
final = spark.sql("SELECT team1,team2, team1score,team2score FROM matchinfo1 WHERE stage = 'Final' ORDER BY matchid")
third = spark.sql("SELECT matchid  FROM matchinfo1 WHERE stage = 'Third Position' ORDER BY matchid")

#converting into pandas
df_gs = gs.toPandas()
df_r16 = r16.toPandas()
df_qf = qf.toPandas()
df_sf = sf.toPandas()
df_final = final.toPandas()
df_third = third.toPandas()

for_visualization = pd.DataFrame({
    'Teams': df_gs['TeamName'],
    'Group': df_gs['GroupName'],
    'Pld': df_gs['TotalGamesPlayed'],
    'W': df_gs['Wins'],
    'D': df_gs['Draws'],
    'L': df_gs['Losses'],
    'GF': df_gs['GoalsFor'],
    'GA': df_gs['GoalsAgainst'],
    'GD': df_gs['GoalDifference'],
    'Pts': df_gs['TotalPoints']
})

blank_row = {
    'Teams': '', 
    'Group': '', 
    'Pld': '', 
    'W': '',
    'D': '',
    'L': '',
    'GF': '',
    'GA': '',
    'GD': '',
    'Pts': ''
}

groups = []
for i in range(0, len(for_visualization), 4):
    group = for_visualization.iloc[i:i+4]  
    groups.append(group) 
    if i < len(for_visualization) - 4:
        groups.append(pd.DataFrame([blank_row])) 

visualization_with_space = pd.concat(groups).reset_index(drop=True)
half_table_index = len(visualization_with_space) // 2
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
display(visualization_with_space)
print(half_table_index)

Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
Germany,A,3,2,0,1,8,7,1,6
Hungary,A,3,1,1,1,12,13,-1,4
France,A,3,1,0,2,8,10,-2,3
Portugal,A,3,1,1,1,10,8,2,4
,,,,,,,,,
Slovakia,B,3,1,1,1,10,13,-3,4
Sweden,B,3,2,1,0,16,10,6,7
Spain,B,3,2,0,1,14,10,4,6
Poland,B,3,0,0,3,7,14,-7,0
,,,,,,,,,


14


In [0]:
%python
#for group A,B,C
left_table = pd.DataFrame(columns=['Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
left_table = pd.concat([left_table, pd.DataFrame([blank_row])], ignore_index=True)
left_table = pd.concat([left_table, visualization_with_space.iloc[:half_table_index]]).reset_index(drop=True)

#for group C,D,E
right_table = pd.DataFrame(columns=['Teams','Group','Pld','W','D','L','GF','GA','GD','Pts'])
right_table = pd.concat([right_table, visualization_with_space.iloc[half_table_index +1:]]).reset_index(drop=True)
right_table = pd.concat([pd.DataFrame([blank_row]), right_table], ignore_index=True)

display(left_table)
display(right_table)


Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,
Germany,A,3,2,0,1,8,7,1,6
Hungary,A,3,1,1,1,12,13,-1,4
France,A,3,1,0,2,8,10,-2,3
Portugal,A,3,1,1,1,10,8,2,4
,,,,,,,,,
Slovakia,B,3,1,1,1,10,13,-3,4
Sweden,B,3,2,1,0,16,10,6,7
Spain,B,3,2,0,1,14,10,4,6
Poland,B,3,0,0,3,7,14,-7,0


Teams,Group,Pld,W,D,L,GF,GA,GD,Pts
,,,,,,,,,
Belgium,D,3,1,1,1,10,12,-2,4
Denmark,D,3,3,0,0,16,7,9,9
Finland,D,3,0,1,2,9,16,-7,1
Russia,D,3,1,0,2,11,11,0,3
,,,,,,,,,
Netherlands,E,3,1,1,1,13,9,4,4
Austria,E,3,1,1,1,7,9,-2,4
Ukraine,E,3,0,2,1,6,9,-3,2
North Macedonia,E,3,2,0,1,10,9,1,6


In [0]:
%sql
select *
from matchinfo1
where stage = 'Round of 16'
order by matchid

matchid,team1,team2,team1score,team2score,result,stage
37,Germany,Switzerland,0,4,Switzerland,Round of 16
38,Portugal,Spain,2,3,Spain,Round of 16
39,Sweden,Hungary,5,1,Sweden,Round of 16
40,Turkey,Russia,1,1,Russia,Round of 16
41,Croatia,Italy,4,3,Croatia,Round of 16
42,Belgium,Netherlands,5,3,Belgium,Round of 16
43,North Macedonia,Slovakia,5,3,North Macedonia,Round of 16
44,Belgium,England,4,3,Belgium,Round of 16


In [0]:
%python
visualize_r16 = pd.DataFrame(columns=['Round_of_16', 'rest'])

for _ in range(2):
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

for index, row in df_r16.iterrows():
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['team1']} ({row['team1score']})"]})], ignore_index=True)
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['team2']} ({row['team2score']})"]})], ignore_index=True)

    for _ in range(2):
        visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

visualize_r16['rest'] = ''

splited_r16_index = len(visualize_r16) // 2
print(splited_r16_index)
display(visualize_r16)

left_side_r16 = visualize_r16.iloc[:splited_r16_index + 1].reset_index(drop = True)
right_side_r16 = visualize_r16.iloc[splited_r16_index - 1:].reset_index(drop = True)
display(left_side_r16)
display(right_side_r16)

17


Round_of_16,rest
,
,
Germany (0),
Switzerland (4),
,
,
Portugal (2),
Spain (3),
,
,


Round_of_16,rest
,
,
Germany (0),
Switzerland (4),
,
,
Portugal (2),
Spain (3),
,
,


Round_of_16,rest
,
,
Croatia (4),
Italy (3),
,
,
Belgium (5),
Netherlands (3),
,
,


In [0]:
%python
visualize_qf = pd.DataFrame(columns=['Quater_Final'])

for _ in range(4):
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': ['']})], ignore_index=True)

for index, row in df_qf.iterrows():
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': [f"{row['team1']} ({row['team1score']})"]})], ignore_index=True)
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': [f"{row['team2']} ({row['team2score']})"]})], ignore_index=True)

    for _ in range(6):
        visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'Quater_Final': ['']})], ignore_index=True)
visualize_qf['rest'] = ''
splited_qf_index = len(visualize_qf) // 2
display(visualize_qf)

Quater_Final,rest
,
,
,
,
Sweden (0),
Switzerland (3),
,
,
,
,


In [0]:
%python
left_side_qf = pd.DataFrame(columns=['Quater_Final'])
left_side_qf = pd.concat([left_side_qf,visualize_qf.iloc[:splited_qf_index]]).reset_index(drop=True)
right_side_qf = pd.DataFrame(columns=['Quater_Final'])
right_side_qf = pd.concat([right_side_qf,visualize_qf.iloc[splited_qf_index-2:]]).reset_index(drop=True)


display(left_side_qf)
display(right_side_qf)

Quater_Final,rest
,
,
,
,
Sweden (0),
Switzerland (3),
,
,
,
,


Quater_Final,rest
,
,
,
,
Turkey (0),
Spain (2),
,
,
,
,


In [0]:
%python
visualize_sf = pd.DataFrame(columns=['Semi_Final'])

for _ in range(8):
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': ['']})], ignore_index=True)

for index, row in df_sf.iterrows():
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': [f"{row['team1']} ({row['team1score']})"]})], ignore_index=True)
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': [f"{row['team2']} ({row['team2score']})"]})], ignore_index=True)
    for _ in range(6):
        visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'Semi_Final': ['']})], ignore_index=True)
visualize_sf['rest'] = ''
splited_sf_index = len(visualize_sf) // 2
print(splited_sf_index)
display(visualize_sf)


12


Semi_Final,rest
,
,
,
,
,
,
,
,
Turkey (1),
England (1),


In [0]:
%python
left_side_sf = pd.DataFrame(columns=['Semi_Final'])
left_side_sf = pd.concat([left_side_sf, visualize_sf.iloc[:splited_sf_index]]).reset_index(drop=True)

for _ in range(5):
    left_side_sf = pd.concat([left_side_sf, pd.DataFrame({'Semi_Final': [''], 'rest': ['']})], ignore_index=True)

right_side_sf = pd.DataFrame({'Semi_Final': [''] * 7, 'rest': [''] * 7})
right_side_sf = pd.concat([right_side_sf, visualize_sf.iloc[splited_sf_index+3:]]).reset_index(drop=True)
display(left_side_sf)
display(right_side_sf)
     

Semi_Final,rest
,
,
,
,
,
,
,
,
Turkey (1),
England (1),


Semi_Final,rest
,
,
,
,
,
,
,
,
Sweden (2),
Netherlands (5),


In [0]:
%python

visualize_final = pd.DataFrame(columns=['Final'])

for _ in range(8):
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)

for index, row in df_final.iterrows():
    winner = ''
    
    if row['team1score'] > row['team2score']:
        winner = row['team1']
    else:
        winner = row['team2']

    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['team1']} ({row['team1score']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['team2']} ({row['team2score']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"Winner: {winner}"]})], ignore_index=True)

visualize_final['rest']= ''
display(visualize_final)



Final,rest
,
,
,
,
,
,
,
,
Sweden (1),
England (1),


In [0]:
%python
left_side_qf.columns = [f"{col}q2" for col in left_side_qf.columns]

merged_df = pd.concat([left_side_r16, left_side_qf], axis=1)

left_side_sf.columns = [f"{col}s1" for col in left_side_sf.columns]
merged_df = pd.concat([merged_df, left_side_sf], axis=1)

visualize_final.columns = [f"{col}f" for col in visualize_final.columns]
merged_df = pd.concat([merged_df, visualize_final], axis=1)

right_side_sf.columns = [f"{col}s2" for col in right_side_sf.columns]

merged_df = pd.concat([merged_df, right_side_sf], axis=1)



right_side_qf.columns = [f"{col}q1" for col in right_side_qf.columns]

merged_df = pd.concat([merged_df, right_side_qf], axis=1)

right_side_r16.columns = [f"{col}r2" for col in right_side_r16.columns]

merged_df = pd.concat([merged_df,right_side_r16], axis=1)

left_table.columns = [f"{col}g1" for col in left_table.columns]
right_table.columns = [f"{col}g2" for col in right_table.columns]


merged_df = pd.concat([left_table,merged_df, right_table], axis=1)

merged_df.fillna('', inplace=True)

display(merged_df)

Teamsg1,Groupg1,Pldg1,Wg1,Dg1,Lg1,GFg1,GAg1,GDg1,Ptsg1,Round_of_16,rest,Quater_Finalq2,restq2,Semi_Finals1,rests1,Finalf,restf,Semi_Finals2,rests2,Quater_Finalq1,restq1,Round_of_16r2,restr2,Teamsg2,Groupg2,Pldg2,Wg2,Dg2,Lg2,GFg2,GAg2,GDg2,Ptsg2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Germany,A,3,2,0,1,8,7,1,6,,,,,,,,,,,,,,,Belgium,D,3,1,1,1,10,12,-2,4
Hungary,A,3,1,1,1,12,13,-1,4,Germany (0),,,,,,,,,,,,Croatia (4),,Denmark,D,3,3,0,0,16,7,9,9
France,A,3,1,0,2,8,10,-2,3,Switzerland (4),,,,,,,,,,,,Italy (3),,Finland,D,3,0,1,2,9,16,-7,1
Portugal,A,3,1,1,1,10,8,2,4,,,Sweden (0),,,,,,,,Turkey (0),,,,Russia,D,3,1,0,2,11,11,0,3
,,,,,,,,,,,,Switzerland (3),,,,,,,,Spain (2),,,,,,,,,,,,,
Slovakia,B,3,1,1,1,10,13,-3,4,Portugal (2),,,,,,,,,,,,Belgium (5),,Netherlands,E,3,1,1,1,13,9,4,4
Sweden,B,3,2,1,0,16,10,6,7,Spain (3),,,,,,,,,,,,Netherlands (3),,Austria,E,3,1,1,1,7,9,-2,4
Spain,B,3,2,0,1,14,10,4,6,,,,,Turkey (1),,Sweden (1),,Sweden (2),,,,,,Ukraine,E,3,0,2,1,6,9,-3,2
Poland,B,3,0,0,3,7,14,-7,0,,,,,England (1),,England (1),,Netherlands (5),,,,,,North Macedonia,E,3,2,0,1,10,9,1,6


In [0]:
%python

merged_df = pd.concat([left_table, left_side_r16, left_side_qf, left_side_sf, visualize_final, right_side_sf, right_side_qf, right_side_r16, right_table], axis=1)
merged_df.fillna('', inplace=True)


display(merged_df)


Teamsg1,Groupg1,Pldg1,Wg1,Dg1,Lg1,GFg1,GAg1,GDg1,Ptsg1,Round_of_16,rest,Quater_Finalq2,restq2,Semi_Finals1,rests1,Finalf,restf,Semi_Finals2,rests2,Quater_Finalq1,restq1,Round_of_16r2,restr2,Teamsg2,Groupg2,Pldg2,Wg2,Dg2,Lg2,GFg2,GAg2,GDg2,Ptsg2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Germany,A,3,2,0,1,8,7,1,6,,,,,,,,,,,,,,,Belgium,D,3,1,1,1,10,12,-2,4
Hungary,A,3,1,1,1,12,13,-1,4,Germany (0),,,,,,,,,,,,Croatia (4),,Denmark,D,3,3,0,0,16,7,9,9
France,A,3,1,0,2,8,10,-2,3,Switzerland (4),,,,,,,,,,,,Italy (3),,Finland,D,3,0,1,2,9,16,-7,1
Portugal,A,3,1,1,1,10,8,2,4,,,Sweden (0),,,,,,,,Turkey (0),,,,Russia,D,3,1,0,2,11,11,0,3
,,,,,,,,,,,,Switzerland (3),,,,,,,,Spain (2),,,,,,,,,,,,,
Slovakia,B,3,1,1,1,10,13,-3,4,Portugal (2),,,,,,,,,,,,Belgium (5),,Netherlands,E,3,1,1,1,13,9,4,4
Sweden,B,3,2,1,0,16,10,6,7,Spain (3),,,,,,,,,,,,Netherlands (3),,Austria,E,3,1,1,1,7,9,-2,4
Spain,B,3,2,0,1,14,10,4,6,,,,,Turkey (1),,Sweden (1),,Sweden (2),,,,,,Ukraine,E,3,0,2,1,6,9,-3,2
Poland,B,3,0,0,3,7,14,-7,0,,,,,England (1),,England (1),,Netherlands (5),,,,,,North Macedonia,E,3,2,0,1,10,9,1,6
